In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader

In [2]:
#Obtain dataset
trainset = datasets.FashionMNIST('./../0. Data/',
                               download = True,
                               train = True,
                               transform = T.Compose([
                                   T.ToTensor()
                               ]))

testset = datasets.FashionMNIST('./../0. Data/',
                              download = True,
                              train = False,
                              transform = T.Compose([
                                  T.ToTensor()
                              ]))

#split training data further to training and validation data
train_set, val_set = torch.utils.data.random_split(trainset, [50000, 10000])

#Put in dataloader
train_loader_full = DataLoader(trainset,
                           batch_size=32,
                           shuffle=True)

train_loader = DataLoader(train_set,
                      batch_size = 32,
                      shuffle = True)

val_loader = DataLoader(val_set,
                       batch_size = 32,
                       shuffle = True)

test_loader = DataLoader(testset,
                        batch_size = 32,
                        shuffle = True)

In [3]:
#check for GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
#Create a MLP class defining our neural network
class vanilla_CNN(nn.Module):
    def __init__(self, input_len, output_len):
        super(vanilla_CNN, self).__init__()
        
        #Convvolutional and max pool layers
        self.conv2d_32 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3), padding=1)
        self.batch2d_1 = nn.BatchNorm2d(32)
        self.conv2d_64 =  nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), padding=1)
        self.batch2d_2 = nn.BatchNorm2d(64)
        self.maxp2d_1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        self.conv2d_128 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), padding=1)
        self.batch2d_3 = nn.BatchNorm2d(128)
        self.maxp2d_2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2d_256 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), padding=1)
        self.batch2d_4 = nn.BatchNorm2d(256)
        self.maxp2d_3 = nn.MaxPool2d(kernel_size=2, stride=2)

        #three fully connected layers
        self.fc1 = nn.Linear(in_features=256*3*3, out_features=512)
        self.bn1 = torch.nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(in_features=512, out_features=256)
        self.bn2 = torch.nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(in_features=256, out_features=10)
        
        
    def forward(self, x):
        x = x.view(-1,1,28,28)
        #input_size = 1x28x28, output_size = 32x28x28
        x = F.relu(self.conv2d_32(x))
        #input_size = 32x28x28, output_size = 32x28x28
        x = self.batch2d_1(x)
        #input_size = 32x28x28, output_size = 64x28x28
        x = F.relu(self.conv2d_64(x))
        #input_size = 64x28x28, output_size = 64x28x28
        x = self.batch2d_2(x)
        #input_size = 64x28x28, output_size = 64x15x15
        x = F.relu(self.maxp2d_1(x))
        #input_size = 64x15x15, output_size = 128x15x15
        x = F.relu(self.conv2d_128(x))
        #input_size = 128x15x15, output_size = 128x15x15
        x = self.batch2d_3(x)
        #input_size = 128x15x15, output_size = 128x7x7
        x = F.relu(self.maxp2d_2(x))
        #input_size = 128x7x7, output_size = 256x7x7
        x = F.relu(self.conv2d_256(x))
        #input_size = 256x7x7, output_size = 256x7x7
        x = self.batch2d_4(x)
        #input_size = 256x7x7, output_size = 256x3x3
        x = F.relu(self.maxp2d_3(x))

        #convert image to a one dimentional tensor before feeding to neural network
        x = x.flatten(start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.dropout(x, p=0.3, training=self.training)
        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.fc3(x)
        
        return x
        

In [5]:
#define training function
def train(Model, validate, max_epoch):
    for epoch in range(max_epoch):
        Train_Loss = []
        Val_Loss =[]
        loader = train_loader_full
        
        if(validate):
            loader = train_loader
        
        cnf_tr = torch.zeros(10,10)
        cnf_val = torch.zeros(10,10)
        
        #Train on training data
        for i, sample in enumerate(loader):

            #set model to train mode
            Model.train()
            #set gradiuents to zero
            optimizer.zero_grad()
            #obtain output
            output = Model(sample[0].to(device)).to(device)
            #compute loss
            loss = loss_function(output, sample[1].to(device))
            #compute gradients
            loss.backward()
            #optimize weights
            optimizer.step()
            #record train loss
            Train_Loss.append(loss.item())
            
            with torch.no_grad():
                #calculate output by argmax
                output = torch.argmax(output, 1)
                #update entries in confusion matrix
                for i in range(output.shape[0]):
                    cnf_tr[output[i],sample[1][i]] +=1
            
        if(validate):
            #Evaluate on validation data
            with torch.no_grad():
                #set model to evaluation mode
                Model.eval()
                #evaluate on tvaidation data
                for i, sample in enumerate(val_loader):
                    output = Model(sample[0].to(device))
                    loss = loss_function(output, sample[1].to(device))
                    Val_Loss.append(loss.item())
                    #calculate output by argmax
                    output = torch.argmax(output, 1)
                    #update entries in confusion matrix
                    for i in range(output.shape[0]):
                        cnf_val[output[i],sample[1][i]] +=1
                   
        actual_count = torch.sum(cnf_tr, dim=0)
        correct_pred = torch.tensor([cnf_tr[i,i] for i in range(10)])
        A_tr = (torch.sum(correct_pred)/torch.sum(actual_count)).item()
        
        if(validate):
            actual_count = torch.sum(cnf_val, dim=0)
            correct_pred = torch.tensor([cnf_val[i,i] for i in range(10)])
            A_val = (torch.sum(correct_pred)/torch.sum(actual_count)).item()
        
        #print losses in every epoch
        if(validate):
            print('epoch : ',epoch,'; Train_acc : ', np.round(A_tr,4), '; Val_acc : ', np.round(A_val,4),  
                  '; Train_loss  ',np.round(np.mean(Train_Loss),4),  '; Val_loss  ',np.round(np.mean(Val_Loss),4))
        else:
            print('epoch = ',epoch,'; Train_acc : ', np.round(A_tr,4), '; Train_loss  ',np.round(np.mean(Train_Loss),4))

In [6]:
#Function top evaluate model using performace metrices
def evaluate(cnf):
    actual_count = torch.sum(cnf, dim=0)
    predicted_count = torch.sum(cnf, dim=1)
    correct_pred = torch.tensor([cnf[i,i] for i in range(10)])
    #Precision
    precision = correct_pred/predicted_count
    #Recall
    recall = correct_pred/actual_count
    #F1-Score
    f1_score = 2*precision*recall/(precision+recall)
    #Accuracy
    Accuracy = torch.sum(correct_pred)/torch.sum(actual_count)
    print('\n',pd.DataFrame({'Class':[i for i in range(10)],
                 'Precision' : precision,
                 'Recall' : recall,
                 'F1_Score': f1_score}))
    
    
    print('\nAccuracy  : ', Accuracy.item())

In [7]:
#function to test model
def test(Model):
    Loss = []
    #confusion matrix
    cnf = torch.zeros(10,10)

    #evaluate on test data
    with torch.no_grad():
        #set model to evaluation mode
        Model.eval()
        #evaluate on test data
        for i, sample in enumerate(test_loader):
            output = Model(sample[0].to(device))
            loss = loss_function(output, sample[1].to(device))
            Loss.append(loss.item())
            #calculate output by argmax
            output = torch.argmax(output, 1)
            #update entries in confusion matrix
            for i in range(output.shape[0]):
                cnf[output[i],sample[1][i]] +=1

        #print test loss
        print('Test loss : ', np.mean(Loss))

    #print evaluation summary
    evaluate(cnf)

In [8]:
#define loss function
loss_function = nn.CrossEntropyLoss()

In [9]:
#Create Model
Model = vanilla_CNN(784,10).to(device)
#Define optimizer
optimizer = optim.Adam(Model.parameters())
#train model with validation
train(Model, validate=True, max_epoch=20)

epoch :  0 ; Train_acc :  0.8528 ; Val_acc :  0.8942 ; Train_loss   0.4134 ; Val_loss   0.6838
epoch :  1 ; Train_acc :  0.9002 ; Val_acc :  0.8992 ; Train_loss   0.2774 ; Val_loss   0.2952
epoch :  2 ; Train_acc :  0.9156 ; Val_acc :  0.9152 ; Train_loss   0.2381 ; Val_loss   0.2425
epoch :  3 ; Train_acc :  0.9249 ; Val_acc :  0.9244 ; Train_loss   0.2061 ; Val_loss   0.2122
epoch :  4 ; Train_acc :  0.9355 ; Val_acc :  0.9278 ; Train_loss   0.1799 ; Val_loss   0.2013
epoch :  5 ; Train_acc :  0.9422 ; Val_acc :  0.9266 ; Train_loss   0.1593 ; Val_loss   0.2217
epoch :  6 ; Train_acc :  0.952 ; Val_acc :  0.9297 ; Train_loss   0.1354 ; Val_loss   0.2099
epoch :  7 ; Train_acc :  0.9584 ; Val_acc :  0.9277 ; Train_loss   0.1177 ; Val_loss   0.2348
epoch :  8 ; Train_acc :  0.9648 ; Val_acc :  0.9313 ; Train_loss   0.0978 ; Val_loss   0.2264
epoch :  9 ; Train_acc :  0.9713 ; Val_acc :  0.9305 ; Train_loss   0.0821 ; Val_loss   0.2625
epoch :  10 ; Train_acc :  0.9744 ; Val_acc :  0.93

In [13]:
#Let's train our model for 12 epochs on full training set
#Create Model
Model = vanilla_CNN(784,10).to(device)
#Define optimizer
optimizer = optim.Adam(Model.parameters())
#train
train(Model, validate=False, max_epoch=12)

epoch =  0 ; Train_acc :  0.8594 ; Train_loss   0.3909
epoch =  1 ; Train_acc :  0.9046 ; Train_loss   0.2649
epoch =  2 ; Train_acc :  0.9188 ; Train_loss   0.2264
epoch =  3 ; Train_acc :  0.9305 ; Train_loss   0.1956
epoch =  4 ; Train_acc :  0.9383 ; Train_loss   0.1702
epoch =  5 ; Train_acc :  0.9482 ; Train_loss   0.1459
epoch =  6 ; Train_acc :  0.956 ; Train_loss   0.1216
epoch =  7 ; Train_acc :  0.9631 ; Train_loss   0.1062
epoch =  8 ; Train_acc :  0.9655 ; Train_loss   0.0957
epoch =  9 ; Train_acc :  0.9721 ; Train_loss   0.0792
epoch =  10 ; Train_acc :  0.9751 ; Train_loss   0.0692
epoch =  11 ; Train_acc :  0.9779 ; Train_loss   0.0618


In [14]:
test(Model)

Test loss :  0.26729777438667257

    Class  Precision  Recall  F1_Score
0      0   0.873621   0.871  0.872308
1      1   0.997972   0.984  0.990937
2      2   0.879692   0.914  0.896518
3      3   0.904986   0.962  0.932622
4      4   0.891109   0.892  0.891554
5      5   0.985972   0.984  0.984985
6      6   0.836245   0.766  0.799582
7      7   0.966469   0.980  0.973188
8      8   0.991968   0.988  0.989980
9      9   0.977778   0.968  0.972864

Accuracy  :  0.930899977684021


In [15]:
#Finally, let's save our model
torch.save(Model.state_dict(), './saved_models/vanilla_CNN_FMNIST.pth')

In [17]:
#To Retrieve
Modelx = vanilla_CNN(784,10).to(device)
Modelx.load_state_dict(torch.load('./saved_models/vanilla_CNN_FMNIST.pth'))

<All keys matched successfully>